In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
# Headers set like below:
# User Agent
headers = {
    'authority': 'www.99acres.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': 'https://www.99acres.com/residential-land-in-gurgaon-ffid-page',
    'sec-ch-ua': '"Chromium";v="107", "Not;A=Brand";v="8"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/527.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
}

### Extracting residential Lands

In [ ]:
land = pd.DataFrame()

In [ ]:
import time

# Put start page number and end page number
start = 1 # Starting Page
end = 50 # End Page

csv_file = f"/content/drive/MyDrive/DSMP/Case Studies/Real estate/resedential-Land/resedential_land_gurgaon_data-p{start}-{end}.csv"
pageNumber = start
req=0
while pageNumber < end:
    i=1
    url = f'https://www.99acres.com/residential-land-in-gurgaon-ffid-page-{pageNumber}'
    page = requests.get(url, headers=headers)
    pageSoup = BeautifulSoup(page.content, 'html.parser')
    req+=1
    for soup in pageSoup.select_one('div[data-label="SEARCH"]').select('section[data-hydration-on-demand="true"]'):

    # Extract property name and property sub-name
        try:
            property_name = soup.select_one('a.srpTuple__propertyName').text.strip()
            # Extract link
            link = soup.select_one('a.srpTuple__propertyName')['href']

        except:
            continue

        try:
            society = soup.select_one('#srp_tuple_society_heading').text.strip()
        except:
            society = ''
        # Detail Page
        page = requests.get(link, headers=headers)
        dpageSoup = BeautifulSoup(page.content, 'html.parser')
        req+=1
        try:
            #price Range
            price = dpageSoup.select_one('#pdPrice2').text.strip()
        except:
            price = ''

        # Area
        try:
            area = soup.select_one('#srp_tuple_secondary_area').text.strip()
        except:
            area =''
        # # Area with Type
        # try:
        #     areaWithType = dpageSoup.select_one('#factArea').text.strip()
        # except:
        #     areaWithType = ''

        # Address
        try:
            address = dpageSoup.select_one('#address').text.strip()
        except:
            address = ''
        # Floor Number
        try:
            floorNumAllowed = dpageSoup.select_one('#floorNumLabel').text.strip()
        except:
            floorNumAllowed = ''

        try:
            noOfOpenSides = dpageSoup.select_one('#noOfOpenSides').text.strip()
        except:
            noOfOpenSides = ''

        try:
            possession = dpageSoup.select_one('#agePossessionLbl').text.strip()
        except:
            possession = ''

        # Nearby Locations

        try:
            nearbyLocations = [i.text.strip() for i in dpageSoup.select_one('div.NearByLocation__tagWrap').select('span.NearByLocation__infoText')]
        except:
            nearbyLocations = ''

        # Descriptions
        try:
            description = dpageSoup.select_one('#description').text.strip()
        except:
            description = ''



        # Features

        try:
            features = [i.text.strip() for i in dpageSoup.select('#features')[0].select('li')]
        except:
            features = ''




        # Rating by Features
        try:
            rating = [i.text for i in dpageSoup.select_one('div.review__rightSide>div>ul>li>div').select('div.ratingByFeature__circleWrap')]
        except:
            rating = ''
        # print(top_f)

        try:
        # Property ID
            property_id = dpageSoup.select_one('#Prop_Id').text.strip()
        except:
            property_id = ''

        # Create a dictionary with the given variables
        property_data = {
        'property_name': property_name,
        'link': link,
        'society': society,
        'price': price,
        'areaWithType': area,

        'address': address,
        'floorNumAllowed': floorNumAllowed,
        'noOfOpenSides': noOfOpenSides,
        'possession': possession,
        'nearbyLocations': nearbyLocations,
        'description': description,

        'features': features,
        'rating':rating,
        'property_id': property_id
    }


        temp_df = pd.DataFrame.from_records([property_data])
        # print(temp_df)
        land = pd.concat([land, temp_df], ignore_index=True)
        i += 1

        if os.path.isfile(csv_file):
        # Append DataFrame to the existing file without header
            temp_df.to_csv(csv_file, mode='a', header=False, index=False)
        else:
            # Write DataFrame to the file with header
            temp_df.to_csv(csv_file, mode='a', header=True, index=False)




    print(f'{pageNumber} -> {i}')
    pageNumber += 1

    if req%4==0:
        time.sleep(10)
    if req %20==0:
        time.sleep(20)

100 -> 26
101 -> 26
102 -> 26
103 -> 26
104 -> 26
105 -> 26
106 -> 26
107 -> 26
108 -> 26
109 -> 26
110 -> 26
111 -> 26


AttributeError: ignored

In [ ]:
land.to_csv('/content/drive/MyDrive/DSMP/Case Studies/Real estate/resedential-Land/resedential_land_gurgaon_data-p1-100_2122rows.csv', index=False)

In [ ]:


def combine_csv_files(folder_path, combined_file_path):
    combined_data = pd.DataFrame()  # Create an empty DataFrame to hold the combined data

    # Iterate through all CSV files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            print('file_path')
            # Read the data from the current CSV file
            df = pd.read_csv(file_path)

            # Append the data to the combined DataFrame
            combined_data = combined_data.append(df, ignore_index=True)

            # Delete the original CSV file
            os.remove(file_path)

    # Save the combined data to a new CSV file
    combined_data.to_csv(combined_file_path, index=False)

# Example usage:
folder_path = '/content/drive/MyDrive/DSMP/Case Studies/Real estate/resedential-Land'  # Replace with the actual folder path
combined_file_path = '/content/drive/MyDrive/DSMP/Case Studies/Real estate/resedential-Land/residential-land.csv'  # Replace with the desired combined file path

combine_csv_files(folder_path, combined_file_path)


file_path
file_path


<ipython-input-5-a103866faa77>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)
<ipython-input-5-a103866faa77>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(df, ignore_index=True)


In [ ]:
pd.read_csv(combined_file_path)

property_name  \
0            Residential land / Plot in Sector-33 Sohna   
1          Residential land / Plot in Sector 93 Gurgaon   
2          Residential land / Plot in Sector 63 Gurgaon   
3          Residential land / Plot in Sector 22 Gurgaon   
4     Residential land / Plot in Block K1 New Palam ...   
...                                                 ...   
2417                   Residential land / Plot in Sohna   
2418                   Residential land / Plot in Sohna   
2419           Residential land / Plot in Garhi Harsaru   
2420              Residential land / Plot in New Colony   
2421       Residential land / Plot in Sector 52 Gurgaon   

                                                   link  \
0     https://www.99acres.com/residential-land-plot-...   
1     https://www.99acres.com/residential-land-plot-...   
2     https://www.99acres.com/residential-land-plot-...   
3     https://www.99acres.com/residential-land-plot-...   
4     https://www.99acres.com/residential-land-plot-...   
...                                                 ...   
2417  https://www.99acres.com/residential-land-plot-...   
2418  https://www.99acres.com/residential-land-plot-...   
2419  https://www.99acres.com/residential-land-plot-...   
2420  https://www.99acres.com/residential-land-plot-...   
2421  https://www.99acres.com/residential-land-plot-...   

                                      society       price  \
0     Central Park Flower Valley Mikasa Plots  3.75 Crore   
1                           ROF Insignia Park  2.23 Crore   
2                 Adani Brahma Samsara Vilasa  8.75 Crore   
3                                         NaN      90 Lac   
4                                         NaN   1.2 Crore   
...                                       ...         ...   
2417                            Corona Greens      75 Lac   
2418                  Supertech Sambhav Homes      50 Lac   
2419                                      NaN      80 Lac   
2420                                      NaN     4 Crore   
2421                                      NaN  4.79 Crore   

               areaWithType  \
0     (209 sq.m.) Plot Area   
1      (17 sq.m.) Plot Area   
2     (286 sq.m.) Plot Area   
3      (42 sq.m.) Plot Area   
4     (167 sq.m.) Plot Area   
...                     ...   
2417   (94 sq.m.) Plot Area   
2418  (130 sq.m.) Plot Area   
2419  (148 sq.m.) Plot Area   
2420  (167 sq.m.) Plot Area   
2421  (286 sq.m.) Plot Area   

                                                address floorNumAllowed  \
0                     Sector-33 Sohna, Gurgaon, Haryana             NaN   
1        Sector 93, Sector 93 Gurgaon, Gurgaon, Haryana             NaN   
2                   Sector 63 Gurgaon, Gurgaon, Haryana             NaN   
3                   Sector 22 Gurgaon, Gurgaon, Haryana             NaN   
4     Block K1 New Palam Vihar Phase 1, Gurgaon, Har...        4 Floors   
...                                                 ...             ...   
2417                            Sohna, Gurgaon, Haryana        4 Floors   
2418                            Sohna, Gurgaon, Haryana        4 Floors   
2419  Near Railway Statiin, Garhi Harsaru, Gurgaon, ...             NaN   
2420  New Colony, Sector 7, New Colony, Gurgaon, Har...             NaN   
2421   Plot No 355, Sector 52 Gurgaon, Gurgaon, Haryana             NaN   

      noOfOpenSides possession  \
0               1.0        NaN   
1               1.0  undefined   
2               2.0        NaN   
3               2.0  Immediate   
4               2.0  Immediate   
...             ...        ...   
2417            1.0  Immediate   
2418            1.0  Immediate   
2419            1.0  undefined   
2420            1.0  undefined   
2421            1.0  undefined   

                                        nearbyLocations  \
0     ['Rapid Metro Sector 55-56', 'CBR Cricket Grou...   
1     ['Newtown Square Mall', 'Dwarka Expy', 'Yaduva...   
2     ['Rapid Metro Station Sec